In [1]:
%cd ../

/Users/kristophermiltiadou/Documents/UniWork/Cambridge/Thesis/CODE/iREC


In [4]:
import numpy as np
import pandas as pd
np.random.seed(0)
#!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data" --no-check-certificate
!curl -O "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv" 
data = pd.read_csv('winequality-red.csv', header=1, delimiter=';').values
data = data[np.random.permutation(np.arange(len(data)))]
data.shape

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84199  100 84199    0     0  69471      0  0:00:01  0:00:01 --:--:-- 69471


(1598, 12)

In [5]:
import torch
import pyro
from torch import nn
import pyro.distributions as dist
from pyro.infer import HMC, MCMC, SVI, NUTS, TraceMeanField_ELBO
from pyro import poutine
from sklearn.datasets import load_boston
import numpy as np
import torch.nn.functional as F
from tqdm.notebook import trange
from rec.utils import kl_estimate_with_mc

In [69]:
x_ = data[:, :-1]
y_ = data[:, -1]
N_tr = int(x_.shape[0] * 0.5)
N_val = x_.shape[0] - N_tr
a = np.arange(x_.shape[0])
train_index = np.random.choice(a, size=N_tr, replace=False)
val_index = np.delete(a, train_index, axis=0)
x_train = x_[train_index]
y_train = y_[train_index]
x_val = x_[val_index][:]
y_val = y_[val_index][:]
x_m = x_train.mean(0)
x_s = x_train.std(0)
x_train = (x_train - x_m) / x_s
x_val = (x_val - x_m) / x_s

D_in = x_train.shape[1]
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_val = torch.FloatTensor(x_val)
y_val = torch.FloatTensor(y_val)

In [95]:
def regression_model(x, y=None, weight_samples=None, in_size=D_in, num_nodes=10, out_size=1, ELBO_BETA=1.):
    # sample vector of weights for regression
    total_weights = (in_size + 1) * num_nodes + (num_nodes + 1) * num_nodes + (num_nodes + 1) * out_size
    # sample params
    with poutine.scale(scale=ELBO_BETA):
        params = pyro.sample("params", dist.Normal(torch.zeros(total_weights + 1), 1.).to_event(1))
    weights, rho = params[:-1], params[-1]

    idx = 0
    fc1_weights = weights[idx: idx + in_size * num_nodes].reshape(num_nodes, in_size)
    idx += in_size * num_nodes
    fc1_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc2_weights = weights[idx: idx + num_nodes * num_nodes].reshape(num_nodes, num_nodes)
    idx += num_nodes * num_nodes
    fc2_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc3_weights = weights[idx: idx + num_nodes * out_size].reshape(out_size, num_nodes)
    idx += num_nodes * out_size
    fc3_bias = weights[idx: idx + out_size].reshape(out_size)
    idx += out_size

    assert idx == total_weights, "Something wrong with number of weights!"

    # compute forward pass
    batch_shape = x.shape[0]
    x = torch.einsum("ij, kj -> ki", fc1_weights, x) + fc1_bias[None].repeat(batch_shape, 1)
    x = torch.tanh(x)

    x = torch.einsum("ij, kj -> ki", fc2_weights, x) + fc2_bias[None].repeat(batch_shape, 1)
    x = torch.tanh(x)

    x = torch.einsum("ij, kj -> ki", fc3_weights, x) + fc3_bias[None].repeat(batch_shape, 1)
    mu = x.squeeze()

    with pyro.plate("data", x.shape[0]):
        obs = pyro.sample("obs", dist.Normal(mu, F.softplus(rho)), obs=y)
    return mu


def KDE_guide(x, y=None, weight_samples=None, in_size=D_in, num_nodes=10, out_size=1, ELBO_BETA=None):
    total_weights = (in_size + 1) * num_nodes + (num_nodes + 1) * num_nodes + (num_nodes + 1) * out_size
    iso_noise = pyro.param("iso_noise", torch.tensor(1e-3), constraint=dist.constraints.positive)
    assignment = dist.Categorical(probs=torch.ones(weight_samples.shape[0])).sample()

    # sample assigmnent
    with poutine.scale(scale=ELBO_BETA):
        params = pyro.sample("params", dist.Normal(weight_samples[assignment], iso_noise).to_event(1))

    weights, rho = params[:-1], params[-1]
    idx = 0
    fc1_weights = weights[idx: idx + in_size * num_nodes].reshape(num_nodes, in_size)
    idx += in_size * num_nodes
    fc1_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc2_weights = weights[idx: idx + num_nodes * num_nodes].reshape(num_nodes, num_nodes)
    idx += num_nodes * num_nodes
    fc2_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc3_weights = weights[idx: idx + num_nodes * out_size].reshape(out_size, num_nodes)
    idx += num_nodes * out_size
    fc3_bias = weights[idx: idx + out_size].reshape(out_size)
    idx += out_size

    assert idx == total_weights, "Something wrong with number of weights!"

    # compute forward pass
    batch_shape = x.shape[0]
    x = torch.einsum("ij, kj -> ki", fc1_weights, x) + fc1_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc2_weights, x) + fc2_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc3_weights, x) + fc3_bias[None].repeat(batch_shape, 1)
    mu = x.squeeze()


def make_empirical_gmm(samples, num_nodes):
    rho_noise = samples['params'][:, -1]
    noise = F.softplus(rho_noise)
    preds_dict = Predictive(regression_model, samples, return_sites=['obs'])(x_val, None, num_nodes=num_nodes)
    preds = preds_dict['obs']
    mix = dist.Categorical(torch.ones(preds.shape[0]))
    comp = dist.Normal(loc=preds.squeeze().permute(1, 0), scale=noise)
    gmm = dist.MixtureSameFamily(mix, comp)
    return gmm


In [103]:
pyro.set_rng_seed(10)
ELBO_BETA = 1.

in_size = x_train.shape[-1]
out_size = 1
num_nodes = 2

# run HMC
kernel = HMC(regression_model, step_size=0.001, num_steps=5, target_accept_prob=0.8)
#nuts_kernel = NUTS(regression_model, step_size=0.001, target_accept_prob=0.65, max_tree_depth=5)
mcmc = MCMC(kernel, num_samples=1000, warmup_steps=5000, num_chains=1)
mcmc.run(x_train, y_train, ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in)

full_samples = mcmc.get_samples(50)
from pyro.infer import Predictive
pred = Predictive(regression_model, full_samples, return_sites=['obs'])(x_val, None, 
                                                                        num_nodes=num_nodes, in_size=D_in)
HMC_RMSE = ((pred['obs'].mean(0) - y_val) ** 2).mean().sqrt()

optimizer = pyro.optim.Adam({"lr": 5e-3})

# train KDE
svi = SVI(regression_model, KDE_guide, optimizer, loss=TraceMeanField_ELBO())

num_iterations = 2500
pyro.clear_param_store()
pbar = trange(num_iterations)
losses = []
for j in pbar:
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train, full_samples['params'], ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in)
    losses.append(loss)
    pbar.set_description("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_train)))

kde_noise = pyro.param("iso_noise")
flattened_params = full_samples['params']
kde_mix = dist.Categorical(probs=torch.ones(flattened_params.shape[0]))
kde_comps = dist.MultivariateNormal(loc=flattened_params,
                                    covariance_matrix=kde_noise * torch.eye(flattened_params.shape[-1]))
kde = dist.MixtureSameFamily(kde_mix, kde_comps)
prior = dist.MultivariateNormal(loc=torch.ones_like(flattened_params[0]),
                                covariance_matrix=torch.eye(flattened_params[0].shape[-1]))
kl_kde_prior = kl_estimate_with_mc(kde, prior)
kde_sample = kde.sample((1000,))
kde_samples = {"params" : kde_sample}
kde_pred = Predictive(regression_model, kde_samples, return_sites=['obs'])(x_val, None, num_nodes=num_nodes,
                                                                           in_size=D_in)
KDE_RMSE = ((kde_pred['obs'].mean(0) - y_val) ** 2).mean().sqrt()

# train Factored Gaussian approx
from pyro.infer.autoguide import AutoDiagonalNormal
guide = AutoDiagonalNormal(regression_model)
svi = SVI(regression_model, guide, optimizer, loss=TraceMeanField_ELBO())
num_iterations = 2500
pyro.clear_param_store()
pbar = trange(num_iterations)
losses = []
for j in pbar:
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train, ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in)
    losses.append(loss)
    pbar.set_description("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_train)))
guide.requires_grad_(False)

params = []
for name, value in pyro.get_param_store().items():
    params.append(pyro.param(name))

means, stds = params
variational_posterior = dist.MultivariateNormal(loc=means, covariance_matrix=torch.diag(stds ** 2))
variational_sample = variational_posterior.sample((1000,))
variational_samples = {"params" : variational_sample}
kl_var_prior = kl_estimate_with_mc(variational_posterior, prior)
var_pred = Predictive(regression_model, variational_samples, return_sites=['obs'])(x_val, None, num_nodes=num_nodes,
                                                                                  in_size=D_in)
VAR_RMSE = ((var_pred['obs'].mean(0) - y_val) ** 2).mean().sqrt()

hmc_gmm = make_empirical_gmm(full_samples, num_nodes)
kde_gmm = make_empirical_gmm(kde_samples, num_nodes)
var_gmm = make_empirical_gmm(variational_samples, num_nodes)
print(f"The final KLs are: KDE {kl_kde_prior}, VAR {kl_var_prior}\n"
      f"The final RMSE are: HMC {HMC_RMSE}, KDE {KDE_RMSE}, VAR {VAR_RMSE}\n"
      f"The final LLs are: HMC {hmc_gmm.log_prob(y_val).sum()}, KDE {kde_gmm.log_prob(y_val).sum()}, VAR {var_gmm.log_prob(y_val).sum()}.")

Sample: 100%|██████████| 6000/6000 [00:12, 462.91it/s, step size=2.85e-02, acc. prob=0.881]


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

The final KLs are: KDE 81.74331665039062, VAR 87.7645034790039
The final RMSE are: HMC 0.667312502861023, KDE 0.6542137861251831, VAR 0.6935590505599976
The final LLs are: HMC -873.2778930664062, KDE -967.4949951171875, VAR -1024.2880859375.


In [104]:
kde_noise

tensor(0.0378, grad_fn=<AddBackward0>)

In [101]:
# train Gaussian approx
from pyro.infer.autoguide import AutoMultivariateNormal
full_guide = AutoMultivariateNormal(regression_model)
svi = SVI(regression_model, full_guide, optimizer, loss=TraceMeanField_ELBO())
num_iterations = 2500
pyro.clear_param_store()
pbar = trange(num_iterations)
optimizer = pyro.optim.Adam({"lr": 1e-9})
losses = []
for j in pbar:
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train, ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in)
    losses.append(loss)
    pbar.set_description("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_train)))
guide.requires_grad_(False)

params = []
for name, value in pyro.get_param_store().items():
    params.append(pyro.param(name))

means, scale_tril = params
gauss_posterior = dist.MultivariateNormal(loc=means, scale_tril=scale_tril)
gauss_sample = gauss_posterior.sample((1000,))
gauss_samples = {"params" : gauss_sample}
kl_gaus_prior = kl_estimate_with_mc(gauss_posterior, prior)
gauss_pred = Predictive(regression_model, gauss_samples, return_sites=['obs'])(x_val, None, num_nodes=num_nodes)
GAUSS_RMSE = ((gauss_pred['obs'].mean(0) - y_val) ** 2).mean().sqrt()

  0%|          | 0/2500 [00:00<?, ?it/s]

In [50]:
GAUSS_RMSE

tensor(0.6533)

In [51]:
gauss_gmm = make_empirical_gmm(gauss_samples, num_nodes)

In [52]:
gauss_gmm.log_prob(y_val).sum()

tensor(-461.5778)